In [22]:
#모듈 로딩

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [23]:
# 데이터 준비
# 확장자 : csv, 분리자 : 세미콜론(;), 데이터 타입 : float32
wine = pd.read_csv('../datas/winequality-white.csv', sep=';', header=1)

In [24]:
# 데이터 확인
wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [32]:
target = wine[wine.columns[-1]]
feature = wine[wine.columns[:-1]]

xtrain, xtest, ytrain, ytest = train_test_split(target, feature)

In [45]:
#훈련 데이터

xtrain, xtest, ytrain, ytest = np.array(xtrain), np.array(ytrain), np.array(ytest), np.array(xtest)
xtrain, xtest, ytrain, ytest = torch.from_numpy(xtrain), torch.from_numpy(ytrain), torch.from_numpy(ytest), torch.from_numpy(xtest)

xtrain = xtrain.type(torch.float32)
ytrain = ytrain.type(torch.float32)
xtest = xtest.type(torch.float32)
ytest = ytest.type(torch.float32)


In [46]:
# 모델 설계하기
md = nn.Sequential(nn.Linear(2,1), nn.Sigmoid())

#optimizer 설정
opt = optim.SGD(md.parameters(), lr=1)

In [47]:
# 학습
def train_md():
    nb_epochs = 1000
    for epoch in range(nb_epochs+1):
        hypothesis = md(xtrain)
        cost = F.binary_cross_entropy(hypothesis, ytrain)
        
        opt.zero_grad()
        cost.backward()
        opt.step()
        
        if epoch%10 ==0:
            prediction = hypothesis>=torch.FloatTensor([0.5])
            correct_prediction = prediction.float() == ytrain
            accuracy = correct_prediction.sum().item()/len(correct_prediction)
            
            print(f'Epoch: {epoch:4d}/{nb_epochs} Cost:{cost.item():.6f} Accuracy{accuracy * 100:2.2f"}%')

In [48]:
train_md()

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x3673 and 2x1)

In [25]:
# DataFrame -> Numpy array변환

wine_array = np.array(wine)

# array -> tensor 변환
wine_tensor = torch.from_numpy(wine_array)

In [26]:
wine_tensor

tensor([[ 7.0000,  0.2700,  0.3600,  ...,  0.4500,  8.8000,  6.0000],
        [ 6.3000,  0.3000,  0.3400,  ...,  0.4900,  9.5000,  6.0000],
        [ 8.1000,  0.2800,  0.4000,  ...,  0.4400, 10.1000,  6.0000],
        ...,
        [ 6.5000,  0.2400,  0.1900,  ...,  0.4600,  9.4000,  6.0000],
        [ 5.5000,  0.2900,  0.3000,  ...,  0.3800, 12.8000,  7.0000],
        [ 6.0000,  0.2100,  0.3800,  ...,  0.3200, 11.8000,  6.0000]],
       dtype=torch.float64)

In [27]:
print(wine_tensor.shape, wine_tensor.ndim)

torch.Size([4898, 12]) 2


In [30]:
target = wine_tensor[-1]
feature = wine_tensor[:-1]

xtrain, xtest, ytrain, ytest = train_test_split(target, feature)

ValueError: Found input variables with inconsistent numbers of samples: [12, 4897]